# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2019
topic: str = "QUÍMICA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [4]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,"1. Sobre una disolución que contiene Ti (IV), Fe"
3,"(III), Bi (III) y Ni (II), se añade cloruro am..."
4,sólido y disolución concentrada de amoniaco
5,para separar los cationes de los grupos 5º y 6º
6,de la marcha analítica del carbonato sódico.
7,¿Cuál de los cationes formará complejo amo-
8,niacal?:
9,


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,4,38,1.0,75,2.0,112,3.0,149,3
1,2,1,39,2.0,76,3.0,113,1.0,150,4
2,3,2,40,1.0,77,4.0,114,2.0,151,2
3,4,1,41,4.0,78,2.0,115,3.0,152,4
4,5,3,42,1.0,79,3.0,116,4.0,153,3
5,6,1,43,2.0,80,1.0,117,3.0,154,1
6,7,2,44,1.0,81,4.0,118,2.0,155,2
7,8,3,45,3.0,82,3.0,119,3.0,156,2
8,9,4,46,NaN,83,2.0,120,1.0,157,3
9,10,3,47,1.0,84,NaN,121,3.0,158,4


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [6]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [7]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [8]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [9]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [10]:
raw_exam_df.head(10)

0     1. Sobre una disolución que contiene Ti (IV), Fe
1    (III), Bi (III) y Ni (II), se añade cloruro am...
2          sólido y disolución concentrada de amoniaco
3      para separar los cationes de los grupos 5º y 6º
4         de la marcha analítica del carbonato sódico.
5          ¿Cuál de los cationes formará complejo amo-
6                                             niacal?:
7                                          1. Ti (IV).
8                                         2. Fe (III).
9                                         3. Bi (III).
Name: text, dtype: object

In [11]:
raw_exam_df.tail(10)

1596                          tura para cada sustancia.
1597    185. El balance de masas de una solución acuosa
1598                      saturada de la sal Ag3PO4 es:
1599      1. 3· [Ag+] = [PO43-] + [HPO42-] + [H2PO4-] +
1600                                            [H3PO4]
1601      2. [Ag+] = 3·([PO43-] + [HPO42-] + [H2PO4-] +
1602                                           [H3PO4])
1603          3. [Ag+] + [H+] = 3·([PO43-] + [HPO42-] +
1604                                  [H2PO4-]) + [OH-]
1605                                4. [Ag+] = [PO43-]3
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [12]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [13]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Sobre una disolución que contiene Ti (IV), ...
1                                          1. Ti (IV).
2                                         2. Fe (III).
3                                         3. Bi (III).
4                                          4. Ni (II).
5    2. El empleo de la energía de microondas para ...
6                      1. Su mayor rapidez y eficacia.
7                           2. Su menor coste inicial.
8    3. Que no es necesaria la etapa de filtración ...
9    4. Que es independiente de la polaridad del ex...
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [14]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 15
Number of rows not ending with '.', ':' or '?' = 33
[87, 186, 188, 190, 192, 221, 260, 268, 277, 282, 338, 344, 478, 602, 606, 712, 713, 714, 715, 761, 773, 774, 775, 776, 822, 839, 910, 925, 928, 936, 937, 938, 939]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [15]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [16]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [17]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [186, 188, 190, 192, 221, 260, 268, 282, 338, 344, 606, 761, 822, 925, 928]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 87:
85    18. ¿Cuál de las siguientes afirmaciones NO es...
86    1. Cuando la concentración de la especie absor...
87    2. Cuando la especie absorbente sufre reaccion...
88    3. La presencia de radiación parásita en el in...
89    4. Cuando el espesor de la cubeta utilizada pa...
Name: text, dtype: object
Id to fix 186:
184    4. tButilmetil éter (por reacción de tBuOK + M...
185    38. ¿Cuál de las siguientes frases, referentes...
186           1. pKa(C6H5-NH3+) = 4.6; pKa(C6H13-NH3+) =
187                                                11.2.
188          2. pKa(C6H5-NH3+) = 11.2; pKa(C6H13-NH3+) =
Name: text, dtype: object
Id to fix 188:
186     1. pKa(C6H5-NH3+) = 4.6; pKa(C6H13-NH3+) =
187                                          11.2.
188    2. pKa(C6H5-NH3+) = 11.2; pKa(C6H13-NH3+) =
189                                           4.6.
190     3. pKa(C6H5-NH3+) = 4.6; pKa(C6H13-NH3+) =
Name: text, dtype: object
Id to fix 190:
188    2. pKa(C6H5-NH3+) = 11.2; pKa(C6H13-

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [18]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 925
925


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [19]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [20]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [21]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [22]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [23]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,"1. Sobre una disolución que contiene Ti (IV), ...",1. Ti (IV).,2. Fe (III).,3. Bi (III).,4. Ni (II).
1,2. El empleo de la energía de microondas para ...,1. Su mayor rapidez y eficacia.,2. Su menor coste inicial.,3. Que no es necesaria la etapa de filtración ...,4. Que es independiente de la polaridad del ex...
2,3. De los siguientes parámetros indique cuál n...,1. El orden de difracción.,2. La longitud de onda de la radiación.,3. El tamaño de las estrías o surcos de la red.,4. La distancia focal del monocromador.
3,4. Indique el pH de una solución de cianato de...,1. 6.36.,2. 9.89.,3. 4.10.,4. 7.00.
4,5. La eficiencia de las columnas empleadas en ...,1. A medida que disminuye el número de platos ...,2. A medida que la altura de plato se hace mayor.,3. A medida que aumenta el número de platos y ...,4. A medida que disminuye la longitud de la co...
5,6. La voltamperometría cíclica:,1. Es la técnica electroanalítica más eficient...,2. Se caracteriza porque el experimento se rea...,3. Es una variación de la polarografía diferen...,4. El electrodo indicador es siempre un electr...
6,7. ¿Cuál de las siguientes afirmaciones acerca...,1. Es una técnica útil para el análisis de bio...,2. La ionización por electronebulización se ll...,3. En el proceso de electronebulización se pro...,4. Se puede adaptar fácilmente a la introducci...
7,"8. En la técnica de electroforesis capilar, un...",1. Aplicando una diferencia de potencial entre...,2. Aplicando una diferencia de presión a travé...,3. Aplicando una diferencia de potencial entre...,4. Haciendo el vacío en uno de los extremos de...
8,9. Considere la reacción de valoración: Ce4+ +...,1. El ion cerio (IV) se oxida a ion Ce (III).,2. El ion hierro (II) se reduce a ion hierro (...,3. El ion cerio (IV) es el agente reductor.,4. El ion hierro (II) es el agente reductor.
9,10. ¿Cuál es el valor de la tensión de descomp...,1. Es el voltaje o potencial de descomposición...,2. Es el potencial de abandono.,3. Es la diferencia entre el potencial de desc...,4. Es el valor del potencial al cual la intens...


In [24]:
exam_df_pivot.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4
175,"176. En un espectro de masas, ¿A que se denomi...",1. Al pico del ión cuya abundancia es del 0%.,2. Al pico del ión cuya abundancia sea las más...,3. Al pico del ión cuya abundancia sea la más ...,4. Al pico del ión cuya abundancia sea del 50%.
176,177. Una técnica muy utilizada para la separac...,1. Por la diferencia en su peso molecular.,2. Por la deferencia en sus puntos isoeléctricos.,3. Por la diferencia en su relación carga/radio.,4. Por la diferencia en su carga.
177,178. Las hélices  de las proteínas se deben a:,1. La formación de puentes de hidrógeno intram...,2. Una organización al azar.,3. La formación de puentes de hidrógeno interm...,4. A la ausencia de puentes de hidrógeno inter...
178,179. Las enzimas alteran:,1. Las velocidades de reacción y los equilibrios.,2. Las velocidades de reacción pero no los equ...,3. Únicamente los equilibrios de la reacción.,4. La afinidad por los sustratos.
179,180. ¿Cuál de las siguientes biomoléculas cont...,1. Alcohol deshidrogenasa.,2. Ascorbato oxidasa,3. Anhidrasa carbónica.,4. Vitamina B12.
180,181. ¿Cuál de los siguientes iones metálicos p...,1. Mn3+.,2. Mg2+.,3. Cr3+.,4. Co3+.
181,182. ¿Cuál de los siguientes procesos son cata...,1. O2 + 4H+ + 4e- → 2H2O.,2. 2O2- + 2H+ → O2 + H2O2.,3. 2H2O → O2 + 4H+ + 4e-.,4. 2H2O2 → 2H2O + O2.
182,183. Si en una farmacia pedimos alcohol del 96...,1. Alcohol puro.,2. Una mezcla azeotrópica de 96% de alcohol y4...,3. Alcohol cuyo punto de ebullición es de 96 ºC.,4. Una mezcla azeotrópica de 96% de alcohol y4...
183,184. El punto crítico de una sustancia pura:,1. Es la temperatura que hay que alcanzar para...,2. Depende de la temperatura a la que se deter...,3. Depende de la presión a la que se determine.,4. Es un punto invariante de presión y tempera...
184,185. El balance de masas de una solución acuos...,1. 3· [Ag+] = [PO43-] + [HPO42-] + [H2PO4-] + ...,2. [Ag+] = 3·([PO43-] + [HPO42-] + [H2PO4-] + ...,3. [Ag+] + [H+] = 3·([PO43-] + [HPO42-] + [H2P...,4. [Ag+] = [PO43-]3


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,4.0
1,2,1.0
2,3,2.0
3,4,1.0
4,5,3.0
5,6,1.0
6,7,2.0
7,8,3.0
8,9,4.0
9,10,3.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,"1. Sobre una disolución que contiene Ti (IV), ...",1. Ti (IV).,2. Fe (III).,3. Bi (III).,4. Ni (II).,4.0
1,2. El empleo de la energía de microondas para ...,1. Su mayor rapidez y eficacia.,2. Su menor coste inicial.,3. Que no es necesaria la etapa de filtración ...,4. Que es independiente de la polaridad del ex...,1.0
2,3. De los siguientes parámetros indique cuál n...,1. El orden de difracción.,2. La longitud de onda de la radiación.,3. El tamaño de las estrías o surcos de la red.,4. La distancia focal del monocromador.,2.0
3,4. Indique el pH de una solución de cianato de...,1. 6.36.,2. 9.89.,3. 4.10.,4. 7.00.,1.0
4,5. La eficiencia de las columnas empleadas en ...,1. A medida que disminuye el número de platos ...,2. A medida que la altura de plato se hace mayor.,3. A medida que aumenta el número de platos y ...,4. A medida que disminuye la longitud de la co...,3.0
5,6. La voltamperometría cíclica:,1. Es la técnica electroanalítica más eficient...,2. Se caracteriza porque el experimento se rea...,3. Es una variación de la polarografía diferen...,4. El electrodo indicador es siempre un electr...,1.0
6,7. ¿Cuál de las siguientes afirmaciones acerca...,1. Es una técnica útil para el análisis de bio...,2. La ionización por electronebulización se ll...,3. En el proceso de electronebulización se pro...,4. Se puede adaptar fácilmente a la introducci...,2.0
7,"8. En la técnica de electroforesis capilar, un...",1. Aplicando una diferencia de potencial entre...,2. Aplicando una diferencia de presión a travé...,3. Aplicando una diferencia de potencial entre...,4. Haciendo el vacío en uno de los extremos de...,3.0
8,9. Considere la reacción de valoración: Ce4+ +...,1. El ion cerio (IV) se oxida a ion Ce (III).,2. El ion hierro (II) se reduce a ion hierro (...,3. El ion cerio (IV) es el agente reductor.,4. El ion hierro (II) es el agente reductor.,4.0
9,10. ¿Cuál es el valor de la tensión de descomp...,1. Es el voltaje o potencial de descomposición...,2. Es el potencial de abandono.,3. Es la diferencia entre el potencial de desc...,4. Es el valor del potencial al cual la intens...,3.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,"1. Sobre una disolución que contiene Ti (IV), ...",1. Ti (IV).,2. Fe (III).,3. Bi (III).,4. Ni (II).,4,2019
1,2. El empleo de la energía de microondas para ...,1. Su mayor rapidez y eficacia.,2. Su menor coste inicial.,3. Que no es necesaria la etapa de filtración ...,4. Que es independiente de la polaridad del ex...,1,2019
2,3. De los siguientes parámetros indique cuál n...,1. El orden de difracción.,2. La longitud de onda de la radiación.,3. El tamaño de las estrías o surcos de la red.,4. La distancia focal del monocromador.,2,2019
3,4. Indique el pH de una solución de cianato de...,1. 6.36.,2. 9.89.,3. 4.10.,4. 7.00.,1,2019
4,5. La eficiencia de las columnas empleadas en ...,1. A medida que disminuye el número de platos ...,2. A medida que la altura de plato se hace mayor.,3. A medida que aumenta el número de platos y ...,4. A medida que disminuye la longitud de la co...,3,2019
5,6. La voltamperometría cíclica:,1. Es la técnica electroanalítica más eficient...,2. Se caracteriza porque el experimento se rea...,3. Es una variación de la polarografía diferen...,4. El electrodo indicador es siempre un electr...,1,2019
6,7. ¿Cuál de las siguientes afirmaciones acerca...,1. Es una técnica útil para el análisis de bio...,2. La ionización por electronebulización se ll...,3. En el proceso de electronebulización se pro...,4. Se puede adaptar fácilmente a la introducci...,2,2019
7,"8. En la técnica de electroforesis capilar, un...",1. Aplicando una diferencia de potencial entre...,2. Aplicando una diferencia de presión a travé...,3. Aplicando una diferencia de potencial entre...,4. Haciendo el vacío en uno de los extremos de...,3,2019
8,9. Considere la reacción de valoración: Ce4+ +...,1. El ion cerio (IV) se oxida a ion Ce (III).,2. El ion hierro (II) se reduce a ion hierro (...,3. El ion cerio (IV) es el agente reductor.,4. El ion hierro (II) es el agente reductor.,4,2019
9,10. ¿Cuál es el valor de la tensión de descomp...,1. Es el voltaje o potencial de descomposición...,2. Es el potencial de abandono.,3. Es la diferencia entre el potencial de desc...,4. Es el valor del potencial al cual la intens...,3,2019


In [34]:
processed_exam_df.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
175,"176. En un espectro de masas, ¿A que se denomi...",1. Al pico del ión cuya abundancia es del 0%.,2. Al pico del ión cuya abundancia sea las más...,3. Al pico del ión cuya abundancia sea la más ...,4. Al pico del ión cuya abundancia sea del 50%.,3,2019
176,177. Una técnica muy utilizada para la separac...,1. Por la diferencia en su peso molecular.,2. Por la deferencia en sus puntos isoeléctricos.,3. Por la diferencia en su relación carga/radio.,4. Por la diferencia en su carga.,1,2019
177,178. Las hélices  de las proteínas se deben a:,1. La formación de puentes de hidrógeno intram...,2. Una organización al azar.,3. La formación de puentes de hidrógeno interm...,4. A la ausencia de puentes de hidrógeno inter...,1,2019
178,179. Las enzimas alteran:,1. Las velocidades de reacción y los equilibrios.,2. Las velocidades de reacción pero no los equ...,3. Únicamente los equilibrios de la reacción.,4. La afinidad por los sustratos.,2,2019
179,180. ¿Cuál de las siguientes biomoléculas cont...,1. Alcohol deshidrogenasa.,2. Ascorbato oxidasa,3. Anhidrasa carbónica.,4. Vitamina B12.,2,2019
180,181. ¿Cuál de los siguientes iones metálicos p...,1. Mn3+.,2. Mg2+.,3. Cr3+.,4. Co3+.,1,2019
181,182. ¿Cuál de los siguientes procesos son cata...,1. O2 + 4H+ + 4e- → 2H2O.,2. 2O2- + 2H+ → O2 + H2O2.,3. 2H2O → O2 + 4H+ + 4e-.,4. 2H2O2 → 2H2O + O2.,4,2019
182,183. Si en una farmacia pedimos alcohol del 96...,1. Alcohol puro.,2. Una mezcla azeotrópica de 96% de alcohol y4...,3. Alcohol cuyo punto de ebullición es de 96 ºC.,4. Una mezcla azeotrópica de 96% de alcohol y4...,4,2019
183,184. El punto crítico de una sustancia pura:,1. Es la temperatura que hay que alcanzar para...,2. Depende de la temperatura a la que se deter...,3. Depende de la presión a la que se determine.,4. Es un punto invariante de presión y tempera...,4,2019
184,185. El balance de masas de una solución acuos...,1. 3· [Ag+] = [PO43-] + [HPO42-] + [H2PO4-] + ...,2. [Ag+] = 3·([PO43-] + [HPO42-] + [H2PO4-] + ...,3. [Ag+] + [H+] = 3·([PO43-] + [HPO42-] + [H2P...,4. [Ag+] = [PO43-]3,2,2019


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [35]:
id_filter = [x for x in range(70, 105, 1)] + [177, 178, 179, 180]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 39
Number of rows in filtered df = 39


**2. Showing the filtered DataFrame**

Using head and tail

In [36]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,71. Las inmunoglobulinas o anticuerpos:,1. Son proteínas solubles que constituyen la b...,2. Son proteínas solubles que constituyen la b...,3. Son proteínas globulares pequeñas y solubles.,4. Una de las más importantes es la Hemoglobina.,2,2019
1,72. ¿Cuál de las siguientes afirmaciones es ci...,1. Se activa cuando la carga energética de las...,2. Se activa con una relación AMP: ATP baja.,3. Activa rutas como la síntesis de ácidos gra...,4. Fosforila sustratos para inhibir la captaci...,1,2019
2,"73. La fenilcetonuria, la causa más frecuente ...",1. Cistationina beta-sintasa.,2. Arginasa.,3. Fenilalanina hidroxilasa.,4. Tirosina aminotransferasa.,3,2019
3,74. ¿Con qué tipo de lipoproteínas se relacion...,1. HDL.,2. LDL.,3. VLDL:,4. Quilomicrones.,1,2019
4,75. La molécula de DNA mitocondrial es:,"1. Circular, de cadena simple.","2. Circular, de cadena doble.","3. Lineal, de cadena simple.","4. Lineal, de cadena doble.",2,2019
5,"76. En una proteína conjugada, el grupo prosté...",1. Es una región globular.,2. Es una subunidad diferente de las demás.,3. Es una parte de la proteína que no está com...,4. Está unido por puentes de hidrógeno.,3,2019
6,77. La enzima reguladora de la biosíntesis del...,1. Colesterol esterasa.,2. Cetotiolasa.,3. Fosfolipasa C.,4. 3-Hidroxi 3-metilglutaril-CoA reductasa.,4,2019
7,78. ¿Qué caracteriza a la enzima lactato deshi...,1. Liberar CO2.,2. Consumir NADH.,3. Tener como sustrato acetil CoA.,4. Permitir que continúe la glucolisis aerobia.,2,2019
8,79. ¿Cuál de las siguientes enzimas tiene como...,1. DNA polimerasa α.,2. DNA polimerasa β.,3. DNA polimerasa γ.,4. DNA polimerasa δ. H3C Cl O + CH3O 80. ¿C...,3,2019
9,1. Cada vuelta de la hélice contiene 3.6 resid...,2. Puede ser paralela o antiparalela.,3. Puede acomodar cualquier tipo de aminoácido...,"4. En esta estructura secundaria, las cadenas ...",81. ¿Cuál de las siguientes afirmaciones acerc...,1,2019


In [37]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
29,1. La carnitina aciltransferasa I.,2. La acil-CoA deshidrogenasa.,3. La enoil-CoA hidratasa.,4. La tiolasa.,101. La gluconeogénesis es una ruta metabólica...,1,2019
30,1. Conduce a la síntesis mitocondrial de gluco...,2. Utiliza exactamente las mismas enzimas que ...,3. Consume una cantidad de energía metabólica ...,"4. Es de naturaleza catabólica, aunque genera ...",102. ¿Qué tipo de inhibidor enzimático AUMENTA...,3,2019
31,1. Inhibidor reversible no competitivo.,2. Inhibidor reversible competitivo.,3. Inhibidor reversible acompetitivo.,4. Inhibidor irreversible.,103. La anhidrasa carbónica:,2,2019
32,1. Es una enzima extracelular.,2. Cataliza la conversión de ácido carbónico e...,3. Tiene un número de recambio muy bajo.,4. Cataliza la reacción de conversión de H2CO3...,104. Respecto de las RNA polimerasas de eucari...,2,2019
33,1. La RNApol I transcribe los precursores de l...,2. La RNApol II transcribe los precursores de ...,3. La RNApol III transcribe los precursores de...,4. La RNApol IV transcribe los precursores de ...,105. Las enzimas denominadas proteasas:,4,2019
34,1. Hidrolizan enlaces glicosídicos en las prot...,2. Catalizan la rotura hidrolítica de enlaces ...,3. Catalizan la rotura hidrolítica de los puen...,4. Hidrolizan enlaces fosfoéster en las proteí...,106. ¿Cuál es el número de coordinación de un ...,2,2019
35,178. Las hélices  de las proteínas se deben a:,1. La formación de puentes de hidrógeno intram...,2. Una organización al azar.,3. La formación de puentes de hidrógeno interm...,4. A la ausencia de puentes de hidrógeno inter...,1,2019
36,179. Las enzimas alteran:,1. Las velocidades de reacción y los equilibrios.,2. Las velocidades de reacción pero no los equ...,3. Únicamente los equilibrios de la reacción.,4. La afinidad por los sustratos.,2,2019
37,180. ¿Cuál de las siguientes biomoléculas cont...,1. Alcohol deshidrogenasa.,2. Ascorbato oxidasa,3. Anhidrasa carbónica.,4. Vitamina B12.,2,2019
38,181. ¿Cuál de los siguientes iones metálicos p...,1. Mn3+.,2. Mg2+.,3. Cr3+.,4. Co3+.,1,2019


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [38]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [39]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [40]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [41]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()